# 04 - Visualizações 2D

Este notebook gera visualizações 2D (PCA, t-SNE, UMAP) para análise qualitativa.


In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd

# Adicionar src ao path
project_root = Path().resolve().parent
sys.path.append(str(project_root))

from src.config import (
    EMBEDDINGS_DIR, REDUCTION_CONFIGS, FIGURES_DIR
)
from src.utils import load_embedding, plot_2d_comparison
